In [1]:
pip install kafka-python

     -------------------------------------- 246.5/246.5 KB 1.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\mvcc1\anaconda3\python.exe -m pip install --upgrade pip' command.


In [12]:
!pip install kafka-python

You should consider upgrading via the 'C:\Users\mvcc1\anaconda3\python.exe -m pip install --upgrade pip' command.


In [25]:
import pandas as pd
from kafka import KafkaConsumer, KafkaProducer
from time import sleep
from json import dumps
import json
from nsepy import get_history
from datetime import date
import talib as tl
from talib import EMA , ADX

In [39]:
producer = KafkaProducer(bootstrap_servers =['43.205.212.33:9092'],
                        value_serializer = lambda x:
                        dumps(x).encode('utf-8'))

In [94]:
producer.send('stock_demo',value = "{'hello' :'World'}")

In [84]:
producer.send('stock_demo',value = "{'surname' :'Malik'}")

In [77]:
producer.send('stock_demo',value = "{'data' :'StockMarket'}")

In [3]:
#Retrieving The Data 

data =  get_history(symbol="RELIANCE",
                            start=date(2007,1,1),
                            end=date(2023,1,27))

In [4]:
data.head()

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
2007-01-02,RELIANCE,EQ,1270.15,1275.0,1285.8,1270.30,1283.75,1280.90,1279.56,1149902,1.471373e+14,NaN,659158,0.5732
2007-01-03,RELIANCE,EQ,1280.90,1285.0,1290.9,1275.35,1286.00,1284.55,1284.00,1489730,1.912809e+14,NaN,713956,0.4793
2007-01-04,RELIANCE,EQ,1284.55,1290.0,1300.0,1277.00,1280.00,1280.50,1284.95,3022423,3.883671e+14,NaN,1442019,0.4771
2007-01-05,RELIANCE,EQ,1280.50,1275.0,1312.0,1275.00,1289.00,1288.15,1296.98,3047126,3.952073e+14,NaN,1184396,0.3887
2007-01-08,RELIANCE,EQ,1288.15,1294.0,1294.0,1266.60,1275.00,1276.85,1276.26,2191355,2.796739e+14,NaN,1091486,0.4981


In [5]:
data = data.reset_index()

In [6]:
#ADDING THE 3 INDICATORS EMA, RSI  AND ADX USING THE TIME PERIOD OF 30 DAYS

data['EMA_LONG'] = tl.EMA(data['Close'], timeperiod = 30)
data['ADX'] = tl.ADX(data['High'],data['Low'], data['Close'], timeperiod=30)
data['RSI'] = tl.RSI(data['Close'],30)

In [7]:
data.sample(1)

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble,EMA_LONG,ADX,RSI
40,2007-03-02,RELIANCE,EQ,1367.75,1371.0,1371.0,1308.0,1317.0,1316.2,1342.13,4747034,6.371127e+14,NaN,2363461,0.4979,1368.667427,NaN,50.160624


In [15]:
data.sample(1).to_dict(orient="records")

[{'Symbol': 'RELIANCE',
  'Series': 'EQ',
  'Prev Close': 2178.7,
  'Open': 2168.5,
  'High': 2231.9,
  'Low': 2168.0,
  'Last': 2193.0,
  'Close': 2191.1,
  'VWAP': 2205.66,
  'Volume': 9002404,
  'Turnover': 1985624455330000.0,
  'Trades': 300037.0,
  'Deliverable Volume': 2389286,
  '%Deliverble': 0.26539999999999997,
  'EMA_LONG': 2063.517743152478,
  'ADX': 12.337620069746556,
  'RSI': 59.198885743057595}]

In [12]:
data.drop(['Date'], axis=1,inplace = True)


In [ ]:
while True:
    dict_stock = data.sample(1).to_dict(orient = "records")[0]
    producer.send('stock_demo',value = dict_stock)
    sleep(5)

In [45]:
producer.flush()